In [1]:
import torch
from torch_geometric import datasets
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader 
import matplotlib.pyplot as plt
from losses import kl_div_vmf, reconstruction_loss
from models import VGAE

%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = datasets.Planetoid(root='/__data/Cora', name='Cora', split='public')

In [6]:
dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [4]:
transform = RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=False, add_negative_train_samples=True)

In [5]:
train_data, val_data, test_data = transform(dataset[0])

In [6]:
print('train data', train_data)
print('val data', val_data)
print('test data', test_data)

dataset[0].edge_index.shape[1] == train_data.edge_label_index.size(1) + val_data.edge_label_index.size(1)  + test_data.edge_label_index.size(1) 

train data Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[8976], edge_label_index=[2, 8976])
val data Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[526], edge_label_index=[2, 526])
test data Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054])


True

In [7]:
feature_dim = dataset[0].x.shape[1]
print('input dim', feature_dim)

input dim 1433


## Check model forward pass

In [15]:
n_vgae = VGAE(input_dim=feature_dim, latent_dim=16, dropout=0.0, latent_distr='normal').to(device)

out_n_vgae, mus_n_vgae, logsigmas2s_n_vgae  = n_vgae(val_data.x.to(device), val_data.edge_label_index.to(device))
print('out', out_n_vgae.shape)
print('mus', mus_n_vgae.shape)
print('logsigmas2s', logsigmas2s_n_vgae.shape)

out torch.Size([2708, 2708])
mus torch.Size([2708, 16])
logsigmas2s torch.Size([2708, 16])


In [8]:
s_vgae = VGAE(input_dim=feature_dim, latent_dim=16, dropout=0.0, latent_distr='vMF').to(device)

out_s_vgae, mus_s_vgae, logkappas_s_vgae  = s_vgae(val_data.x.to(device), val_data.edge_label_index.to(device))
print('out', out_s_vgae.shape)
print('mus', mus_s_vgae.shape)
print('logkappas', logkappas_s_vgae.shape)

out torch.Size([2708, 2708])
mus torch.Size([2708, 16])
logkappas torch.Size([2708, 1])


In [10]:
out_s_vgae

tensor([[ 1.0000,  0.1624,  0.0029,  ..., -0.0420, -0.1526, -0.1205],
        [ 0.1624,  1.0000, -0.2017,  ..., -0.1567, -0.0480, -0.1645],
        [ 0.0029, -0.2017,  1.0000,  ...,  0.2821, -0.1984,  0.4718],
        ...,
        [-0.0420, -0.1567,  0.2821,  ...,  1.0000, -0.0085,  0.0208],
        [-0.1526, -0.0480, -0.1984,  ..., -0.0085,  1.0000, -0.3528],
        [-0.1205, -0.1645,  0.4718,  ...,  0.0208, -0.3528,  1.0000]],
       dtype=torch.float64, grad_fn=<MmBackward0>)

In [9]:
num_epochs = 2
lr = 0.01
optimizer = torch.optim.Adam(s_vgae.parameters(), lr=lr)

## Train $\mathcal{S}$-VGAE

In [12]:
log_loss = []

for epoch in range(num_epochs):
    kl = kl_div_vmf.apply
    
    optimizer.zero_grad()

    output, mus, logkappas = s_vgae(train_data.x.to(device), train_data.edge_label_index.to(device) )
    kappas = torch.exp(logkappas)
    loss = kl(mus, kappas)
    print('loss', loss)
    loss.backward()

    optimizer.step()

    with torch.no_grad():
    #     output_val, mus_val, logkappas_val = s_vgae
        log_loss.append(loss.item())


loss tensor([[0.0283],
        [0.0278],
        [0.0322],
        ...,
        [0.0295],
        [0.0298],
        [0.0307]], grad_fn=<kl_div_vmfBackward>)


RuntimeError: grad can be implicitly created only for scalar outputs

In [18]:
torch.log(torch.tensor(torch.pi))

tensor(1.1447)

In [ ]:
def plot_curves(log_loss):
    fig = plt.figure()
    